In [ ]:
# -*- coding: utf-8 -*-
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as scio

dtype = torch.float
device = torch.device("cuda:0")

In [ ]:
class Poly1(torch.nn.Module):
    def __init__(self,dimension):
        super().__init__()
        self.dim = dimension

    def forward(self, x):
        # return poly(1, self.dim)
        return -torch.log(1 - (-torch.exp(x)))

class Qnet(torch.nn.Module):
    def __init__(self, dimension):
        super().__init__()
        with torch.no_grad():
            self.model = torch.nn.Sequential(torch.nn.Linear(dimension+1, 2**dimension, bias=False, dtype=torch.float32, device=device), 
            Poly1(dimension), 
            torch.nn.Linear(2**dimension, 1, bias=False, dtype=torch.float32,device=device))

        tpp = []
        for i in np.arange(0, 2**dimension, 1):
            tpp.append(list(np.binary_repr(i, width=dimension)))
        
        tpp = np.array(tpp).astype(float)
        S = torch.tensor(np.concatenate(((tpp == 0) * -1 + tpp, np.full((2**dimension, 1), -1)), axis=1), dtype=torch.float32, device=device)
        w3 = (-torch.prod(S, axis=1).T).reshape(1,S.size(0))

        # S[0][0] = -0.75
        # S[1][0] = 0

        self.model[0].weight = torch.nn.Parameter(S)
        self.model[2].weight = torch.nn.Parameter(w3)

    def forward(self, x):
        return self.model(x)

class Intergrator1():
    def __init__(self):
        self.qnet = Qnet(1)

    def apply(self, W1, B1, W2, B2, start, end):
        self.qnet.model[0].weight[0][0] = -end
        self.qnet.model[0].weight[1][0] = -start
        y = torch.cat((W1, B1.reshape(W1.size(0),1)), axis=1)
        res = torch.div(self.qnet(y), torch.prod(W1, axis = 1).reshape(W1.size(0),1)) + end-start
        self.model = torch.nn.Linear(W2.size(1),1, dtype=torch.float32,device=device)
        self.model.weight = torch.nn.Parameter(W2)
        self.model.bias = torch.nn.Parameter(B2*(end-start))#2**self.dim))

        return self.model(res.T)